### Let's try HP Tuning on this too, which can help us later

In [4]:
%%capture
!pip install smdebug torch torchvision tqdm

In [5]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
from smdebug import modes
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import IPython

In [27]:
hyperparameters = {"batch_size": 32, \
                   "learning_rate": "0.005"}
hyperparameters

{'batch_size': 32, 'learning_rate': '0.005'}

In [28]:
role = sagemaker.get_execution_role()

In [29]:
estimator = PyTorch(
    entry_point='model_train_benchmark.py',
    base_job_name='benchmark-bins',
    role=role,
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    framework_version='1.4.0',
    py_version='py3',
    hyperparameters=hyperparameters,
)

In [30]:
os.environ['SM_CHANNEL_TRAINING']='s3://sagemaker-studio-0qajs6acpsz/BinDataset/'
os.environ['SM_MODEL_DIR']='s3://sagemaker-studio-0qajs6acpsz/model/'
os.environ['SM_OUTPUT_DATA_DIR']='s3://sagemaker-studio-0qajs6acpsz/output/'
estimator.fit({"training": "s3://sagemaker-studio-0qajs6acpsz/BinDataset/"}, wait=False)